In [1]:
#creates a directory without throwing an error
import os
def create_dir(dir):
  if not os.path.exists(dir):
    os.makedirs(dir)
    print("Created Directory : ", dir)
  else:
    print("Directory already existed : ", dir)
  return dir
create_dir('generated_samples')

Created Directory :  generated_samples


'generated_samples'

In [2]:
!wget https://www.dropbox.com/s/48wy3kw4e512ax7/logs.rar?dl=0

--2022-02-17 08:50:35--  https://www.dropbox.com/s/48wy3kw4e512ax7/logs.rar?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/48wy3kw4e512ax7/logs.rar [following]
--2022-02-17 08:50:35--  https://www.dropbox.com/s/raw/48wy3kw4e512ax7/logs.rar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucac9aaa1d9cd2e3e09698d7ba0d.dl.dropboxusercontent.com/cd/0/inline/Bf6j_pRqiNCW5U34eBS1e38jZwGiKk8RZPz55ASFpFfbH-s7pVGdlM3KlWxSl_HQUTd7s5S2zaVx2JVltY7YtaMFpxv5f6RWa9kGFFj1j5pnhpMIZGg5ugrUEne-CrTMKVNxuxHwxFs_DTnp2Hg0XOn1/file# [following]
--2022-02-17 08:50:35--  https://ucac9aaa1d9cd2e3e09698d7ba0d.dl.dropboxusercontent.com/cd/0/inline/Bf6j_pRqiNCW5U34eBS1e38jZwGiKk8RZPz55ASFpFfbH-s7pVGdlM3KlWxSl_HQUTd7s5S2zaVx2JVltY7YtaMFpxv5f6RWa

In [20]:
!unrar x /content/logs.rar\?dl\=0


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/logs.rar?dl=0

Creating    logs                                                      OK
Creating    logs/super_resolution                                     OK
Extracting  logs/super_resolution/checkpoint                               0%  OK 
Extracting  logs/super_resolution/model_gs_141k.data-00000-of-00001        0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50% 51%  OK 
Extracting  logs/super_resolution/model_gs_141k.index                     51%  OK 
Extracting  logs/super_resolution/model_gs_141k.meta         

In [23]:
import sys
import os
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
sys.path.append('tools')
sys.path.append('network')
%load_ext autoreload
%autoreload 2
tf.reset_default_graph()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
class HP:
    
    persianvocab='PE !.:،؟ءآئابتثجحخدذرزسشصضطظعغفقلمنهوىًَُِّْپچژکگی'

    embeding_num_units=128
    d=256#Hidden units for text 2 mel according to the paper
    c=512#Hidden units for super resution network according to the paper
    dropout_rate=0
    
    # signal processing
    sr = 22050  # Sampling rate.
    n_fft = 1024  # fft points (samples)
    frame_shift = 0.0125  # seconds
    frame_length = 0.05  # seconds
    hop_length = 254  # samples.
    n_mels = 80  # Number of Mel banks to generate
    power = 1.5  # Exponent for amplifying the predicted magnitude
    n_iter = 150  # Number of inversion iterations
    preemphasis = .97
    sharpening_factor=1.3
    max_db = 100
    min_db=-100
    ref_db = 20
    r = 4 # Reduction factor
    logdir = None
    
    init_learinig_rate=4e-4
    batch_size=4
    Max_Number_Of_Chars = 180 
    Max_Number_Of_MelFrames = 210

In [25]:
from __future__ import print_function, division

import numpy as np
import librosa
import os, copy
from scipy import signal
from scipy.io import wavfile
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pdb

''' 
modified from:
https://github.com/r9y9/deepvoice3_pytorch/blob/master/audio.py
https://github.com/andabi/voice-vector/blob/master/audio.py 
https://github.com/Kyubyong/dc_tts/blob/master/utils.py
'''
def load_spectrograms_npz(fpath,mode):
    fname = os.path.basename(fpath)
    #for some reasons windows started fucking with me :/
    if mode==1:
        mel_path=str(fpath).replace('.npz','_mel.npz').replace('\\','/').replace('//','/').replace("b'",'').replace("'",'')
        mag_path=str(fpath).replace('.npz','_mag.npz').replace('\\','/').replace('//','/').replace("b'",'').replace("'",'')
        mel=np.load(mel_path)
        mag=np.load(mag_path)
        return fname, mel['mel'].astype(np.float32), mag['mag'].astype(np.float32)
    elif mode==2:
        mel_path=str(fpath).replace('.npz','_mel.npz').replace('\\','/').replace('//','/').replace("b'",'').replace("'",'')
        mel=np.load(mel_path)
        return fname, mel['mel'].astype(np.float32)
    
def load_spectrograms(fpath):
    
    fname = os.path.basename(fpath)
    mel, mag = get_spectrograms(fpath)
    t = mel.shape[0]

    # Marginal padding for reduction shape sync.
    num_paddings = HP.r - (t % HP.r) if t % HP.r != 0 else 0
    mel = np.pad(mel, [[0, num_paddings], [0, 0]], mode="constant")
    mag = np.pad(mag, [[0, num_paddings], [0, 0]], mode="constant")

    # Reduction
    mel = mel[::HP.r, :]
    return fname, mel, mag

def get_spectrograms(fpath):

    # Loading sound file
    y, sr = librosa.load(fpath, sr=HP.sr)

    # Trimming
    y, _ = librosa.effects.trim(y)

    # Preemphasis
    y = np.append(y[0], y[1:] - HP.preemphasis * y[:-1]) 

    # stft
    linear = librosa.stft(y=y,
                          n_fft=HP.n_fft,
                          hop_length=HP.hop_length,
                          window='hann')

    # magnitude spectrogram
    mag = np.abs(linear)  # (1+n_fft//2, T)

    # mel spectrogram
    mel_basis = librosa.filters.mel(HP.sr, HP.n_fft, HP.n_mels)  # (F, 1+n_fft//2)
    mel = np.dot(mel_basis, mag)  # (F, t)

    # amplitude to decibel
    mel = _amp_to_db(mel)
    mag = _amp_to_db(mag)

    # normalize
    mel = _normalize(mel)
    mag = _normalize(mag)


    # Transpose
    mel = mel.T.astype(np.float32)  # (T, F)
    mag = mag.T.astype(np.float32)  # (T, 1+n_fft//2)

    return mel, mag

def spectrogram2wav(mag):

    # transpose and ensuring mag
    mag = np.clip(mag.T,0,1)

    # de-normalize, convert back to linear magnitudes
    mag = _db_to_amp(_denormalize(mag)) 

    # apply sharpening factor
    mag = mag ** HP.sharpening_factor

    # wav reconstruction
    wav = griffin_lim(mag)

    # undo-preemphasis
    wav = signal.lfilter([1], [1, -HP.preemphasis], wav)

    # trim out silent portions of signal
    wav, _ = librosa.effects.trim(wav)

    return wav.astype(np.float32)

def griffin_lim(spectrogram):

    X_best = copy.deepcopy(spectrogram)
    for i in range(HP.n_iter):
        X_t = invert_spectrogram(X_best)
        est = librosa.stft(X_t,n_fft=HP.n_fft,hop_length=HP.hop_length,window='hann')
        phase = est / np.maximum(1e-8, np.abs(est))
        X_best = spectrogram * phase
    X_t = invert_spectrogram(X_best)
    y = np.real(X_t)

    return y

def invert_spectrogram(spectrogram):

    return librosa.istft(spectrogram,hop_length=HP.hop_length, window="hann")


def _amp_to_db(x):
    min_level = np.exp(HP.min_db / 20 * np.log(10))
    return 20 * np.log10(np.maximum(min_level, x))

def _db_to_amp(x):
    return np.power(10.0, x * 0.05)

def _normalize(S):

    S = S - HP.min_db 
    S = S/np.max(S)        
    return np.clip(S, 1e-8, 1)

def _denormalize(S):
    S = S*(HP.ref_db-HP.min_db) 
    S = S + HP.min_db           
    return S

def save_wav(wav, path, sr):

    wavfile.write(path, sr, wav) 


In [26]:
import numpy as np
import unicodedata
import re
import os
import codecs
import tensorflow as tf

def load_vocab():
    char2index={char:indx for indx,char in enumerate(HP.persianvocab)}
    index2char={indx:char for indx,char in enumerate(HP.persianvocab)}
    return char2index,index2char

def text_norm(text):
    #remove all the "diacritic"
    text = ''.join(char for char in unicodedata.normalize('NFD', text)
                           if unicodedata.category(char) != 'Mn')
    
    text = text.lower()
    text = re.sub("[^{}]".format(HP.persianvocab), " ", text)
    text = re.sub("[ ]+", " ", text)
    return text

def load_data(text):
    char2index,index2char=load_vocab()
    text+='E'
    return np.asarray([char2index[i] for i in text],np.int32)

def load_data_synthesize(lines,Max_Number_Of_Chars):
    char2index,index2char=load_vocab()
    sents = [text_norm(line) + "E" for line in lines]
    print(sents)
    texts = np.zeros((len(sents), Max_Number_Of_Chars), np.int32)
    for i, sent in enumerate(sents):
        texts[i, :len(sent)] = [char2index[char] for char in sent]
    return texts

def load_data_main(path,text_corpus_name):
    char2index,index2char=load_vocab()
    wavpaths,sentense_lenghts,sentenses=[],[],[]
    lines = codecs.open(os.path.join(path,text_corpus_name), 'r', 'utf-8').readlines()
    for line in lines:
            wname,sentense=line.strip().split('|')
            wpath = os.path.join(path,'wavs', wname + ".wav")
            wavpaths.append(wpath)
            norm_text=text_norm(line)+'E'
            convertedtoindx=[char2index[i] for i in norm_text]
            #i convert it to string because slice_input_producer expects it to be string
            sentenses.append(np.array(convertedtoindx, np.int32).tostring())
            sentense_lenghts.append(len(convertedtoindx))
    return wavpaths,sentense_lenghts,sentenses

def load_data_main_npz(path,text_corpus_name):
    char2index,index2char=load_vocab()
    wavpaths,sentense_lenghts,sentenses=[],[],[]
    lines = codecs.open(os.path.join(path,text_corpus_name), 'r', 'utf-8').readlines()
    for line in lines:
            wname,sentense=line.strip().split('|')
            wpath = os.path.join(path,'pickle', wname + ".npz")
            wavpaths.append(wpath)
            norm_text=text_norm(line)+'E'
            convertedtoindx=[char2index[i] for i in norm_text]
            #i convert it to string because slice_input_producer expects it to be string
            sentenses.append(np.array(convertedtoindx, np.int32).tostring())
            sentense_lenghts.append(len(convertedtoindx))
    return wavpaths,sentense_lenghts,sentenses
     
            
def get_batch(path,text_corpus_name,batch_size,n_mels,n_fft):
    with tf.device('/cpu:0'):
        fpaths, text_lengths, texts = load_data_main(path,text_corpus_name) # list
        maxlen, minlen = max(text_lengths), min(text_lengths)

        num_batch = len(fpaths) // batch_size

        # Create Queues
        fpath, text_length, text = tf.train.slice_input_producer([fpaths, text_lengths, texts], shuffle=True)

        text = tf.decode_raw(text, tf.int32)  # (None,)
            
        parse_func = lambda path: load_spectrograms(path)
        fname, mel, mag = tf.py_func(parse_func, [fpath], [tf.string, tf.float32, tf.float32])  # (None, F)
        
        fname.set_shape(())
        text.set_shape((None,))
        mel.set_shape((None, n_mels))
        mag.set_shape((None, n_fft//2+1))

        # Batching for more details :https://github.com/wcarvalho/jupyter_notebooks/blob/ebe762436e2eea1dff34bbd034898b64e4465fe4/tf.bucket_by_sequence_length/bucketing%20practice.ipynb
        _, (texts, mels, mags, fnames)  = tf.contrib.training.bucket_by_sequence_length(
                                            input_length=text_length,
                                            tensors=[text, mel, mag, fname],
                                            batch_size=batch_size,
                                            bucket_boundaries=[i for i in range(minlen + 1, maxlen - 1, 20)],
                                            num_threads=8,
                                            capacity=batch_size*4,
                                            dynamic_pad=True)

    return  texts, mels, mags, fnames,num_batch

def get_batch_npz(path,text_corpus_name,batch_size,n_mels,n_fft,mode):
    with tf.device('/cpu:0'):
        fpaths, text_lengths, texts = load_data_main_npz(path,text_corpus_name) # list
        maxlen, minlen = max(text_lengths), min(text_lengths)

        num_batch = len(fpaths) // batch_size

        # Create Queues
        fpath, text_length, text = tf.train.slice_input_producer([fpaths, text_lengths, texts], shuffle=True)

        text = tf.decode_raw(text, tf.int32)  # (None,)
        if mode==1:    
            fname, mel, mag = tf.py_func(load_spectrograms_npz, [fpath,mode], [tf.string, tf.float32, tf.float32])  # (None, n_mels)
            fname.set_shape(())
            text.set_shape((None,))
            mel.set_shape((None, n_mels))
            mag.set_shape((None, n_fft//2+1))
        if mode==2:    
            fname, mel = tf.py_func(load_spectrograms_npz, [fpath,mode], [tf.string, tf.float32])  # (None, n_mels)
            fname.set_shape(())
            text.set_shape((None,))
            mel.set_shape((None, n_mels))
            


        # Batching for more details :https://github.com/wcarvalho/jupyter_notebooks/blob/ebe762436e2eea1dff34bbd034898b64e4465fe4/tf.bucket_by_sequence_length/bucketing%20practice.ipynb
        if mode==1:
            _, (mels, mags, fnames)  = tf.contrib.training.bucket_by_sequence_length(
                                            input_length=text_length,
                                            tensors=[mel, mag, fname],
                                            batch_size=batch_size,
                                            bucket_boundaries=[i for i in range(minlen + 1, maxlen - 1, 20)],
                                            num_threads=8,
                                            capacity=batch_size*4,
                                            dynamic_pad=True)
            return  mels, mags, fnames,num_batch
        elif mode==2:
            _, (texts, mels, fnames)  = tf.contrib.training.bucket_by_sequence_length(
                                            input_length=text_length,
                                            tensors=[text, mel, fname],
                                            batch_size=batch_size,
                                            bucket_boundaries=[i for i in range(minlen + 1, maxlen - 1, 20)],
                                            num_threads=8,
                                            capacity=batch_size*4,
                                            dynamic_pad=True)
            return  texts, mels, fnames,num_batch
            
            

In [27]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

#https://www.w3cschool.cn/doc_tensorflow_python/tensorflow_python-tf-layers-conv1d.html
'''He initialization is implemented in variance_scaling_initializer(), but it didnt work for me (loss keeps blowing up) so i used Xavier initialization.i found the following questions useful:

https://stackoverflow.com/questions/43284047/what-is-the-default-kernel-initializer-in-tf-layers-conv2d-and-tf-layers-dense
https://stackoverflow.com/questions/37350131/what-is-the-default-variable-initializer-in-tensorflow
https://datascience.stackexchange.com/questions/13061/when-to-use-he-or-glorot-normal-initialization-over-uniform-init-and-whatare'''
def conv1d(input_tensor,filters,kernel_size,strides,padding,dilation_rate,activation,dropout_rate):
    
    if padding.lower() == "causal":
        # pre-padding for causality
        #for more information :https://github.com/keras-team/keras/issues/8751
        pad_len = (kernel_size - 1) * dilation_rate  # padding size
        input_tensor = tf.pad(input_tensor, [[0, 0], [pad_len, 0], [0, 0]])
        padding = "valid"
        
    if filters==None:
        filters=input_tensor.get_shape().as_list()[-1]
    
    
    output=tf.layers.conv1d(
        input_tensor,
        filters,
        kernel_size,
        strides,
        padding,
        data_format='channels_last',
        dilation_rate=dilation_rate,
        activation=activation,
        use_bias=True,
        kernel_initializer=None,
        bias_initializer=tf.zeros_initializer(),
        kernel_regularizer=None,
        bias_regularizer=None,
        activity_regularizer=None,
        kernel_constraint=None,
        bias_constraint=None,
        trainable=True,
        name=None,
        reuse=None)
    if(dropout_rate>0):
        output =tf.contrib.layers.layer_norm(output,begin_norm_axis=-1)
        output = tf.layers.dropout(output, rate=dropout_rate)
    return output


def highwaynet(input_tensor,filters,kernel_size,strides,padding,dilation_rate,activation,dropout_rate,scope_name):
    with tf.variable_scope(scope_name,reuse=None):
        
            _input_tensor=input_tensor
            if filters==None:
                filters=input_tensor.get_shape().as_list()[-1]
            output=conv1d(input_tensor=input_tensor,filters=2*filters,
                            kernel_size=kernel_size,strides=strides,padding=padding, 
                            dilation_rate=dilation_rate,activation=activation,dropout_rate=dropout_rate)

            
            H1,H2=tf.split(output,2,axis=-1)
            if(dropout_rate>0):
                H1=tf.contrib.layers.layer_norm(H1,begin_norm_axis=-1)
                H2=tf.contrib.layers.layer_norm(H2,begin_norm_axis=-1)
            output=tf.nn.sigmoid(H1)*H2+(1.-tf.nn.sigmoid(H1))*_input_tensor
            if(dropout_rate>0):
                output = tf.layers.dropout(output, rate=dropout_rate)
    return output

#fancy deconvolution
def deconv(input_tensor,filters,kernel_size,strides,padding,activation,dropout_rate):
    
    if filters==None:
        filters=input_tensor.get_shape().as_list()[-1]
        
    #expand inputs dimension to make it suitable for 2d conv     
    input_tensor=tf.expand_dims(input_tensor, 1)
    output=tf.layers.conv2d_transpose(
        input_tensor,
        filters,
        (1,kernel_size),
        (1,strides),
        padding,
        data_format='channels_last',
        activation=activation,
        use_bias=True,
        kernel_initializer=None,
        bias_initializer=tf.zeros_initializer(),
        kernel_regularizer=None,
        bias_regularizer=None,
        activity_regularizer=None,
        kernel_constraint=None,
        bias_constraint=None,
        trainable=True,
        name=None,
        reuse=None)
    
    output = tf.squeeze(output, 1)
    if(dropout_rate>0):
        output =tf.contrib.layers.layer_norm(output,begin_norm_axis=-1)
        output = tf.layers.dropout(output, rate=dropout_rate)
        
    return output

In [28]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

def embeding_layer(inputtextids,vocab_size,emdeding_size,scope_name,padding=False):
    with tf.variable_scope(scope_name,reuse=None):
        table=tf.get_variable(
            "table",
            shape=[vocab_size,emdeding_size],
            dtype=tf.float32,
            initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.1))
        return tf.nn.embedding_lookup(table,inputtextids)
    
def textencoder(embeding_tensor,dropout_rate,num_hidden_layers):
    L1=conv1d(input_tensor=embeding_tensor,filters=num_hidden_layers*2,kernel_size=1,strides=1,padding="SAME",dilation_rate=1,
       activation=tf.nn.relu,dropout_rate=dropout_rate)
    
    L2=conv1d(input_tensor=L1,filters=None,kernel_size=1,strides=1,padding="SAME",dilation_rate=1,
       activation=None,dropout_rate=dropout_rate)
    
    L3=highwaynet(input_tensor=L2,filters=None,kernel_size=3,
                         strides=1,padding="SAME",dilation_rate=1,
                         activation=None,dropout_rate=dropout_rate,scope_name="textencoder_highwaynet_Block"+str(1))
    L4=highwaynet(input_tensor=L3,filters=None,kernel_size=3,
                         strides=1,padding="SAME",dilation_rate=3,
                         activation=None,dropout_rate=dropout_rate,scope_name="textencoder_highwaynet_Block"+str(2))
    L5=highwaynet(input_tensor=L4,filters=None,kernel_size=3,
                         strides=1,padding="SAME",dilation_rate=9,
                         activation=None,dropout_rate=dropout_rate,scope_name="textencoder_highwaynet_Block"+str(3))
    L6=highwaynet(input_tensor=L5,filters=None,kernel_size=3,
                         strides=1,padding="SAME",dilation_rate=27,
                         activation=None,dropout_rate=dropout_rate,scope_name="textencoder_highwaynet_Block"+str(4))
    
    L7=highwaynet(input_tensor=L6,filters=None,kernel_size=3,
                         strides=1,padding="SAME",dilation_rate=1,
                         activation=None,dropout_rate=dropout_rate,scope_name="textencoder_highwaynet_Block"+str(5))
    L8=highwaynet(input_tensor=L7,filters=None,kernel_size=3,
                         strides=1,padding="SAME",dilation_rate=3,
                         activation=None,dropout_rate=dropout_rate,scope_name="textencoder_highwaynet_Block"+str(6))
    L9=highwaynet(input_tensor=L8,filters=None,kernel_size=3,
                         strides=1,padding="SAME",dilation_rate=9,
                         activation=None,dropout_rate=dropout_rate,scope_name="textencoder_highwaynet_Block"+str(7))
    L10=highwaynet(input_tensor=L9,filters=None,kernel_size=3,
                         strides=1,padding="SAME",dilation_rate=27,
                         activation=None,dropout_rate=dropout_rate,scope_name="textencoder_highwaynet_Block"+str(8))
    


    L11=highwaynet(input_tensor=L10,filters=None,kernel_size=3,
                     strides=1,padding="SAME",dilation_rate=1,
                     activation=None,dropout_rate=dropout_rate,scope_name="textencoder_highwaynet_Block"+str(9))

    L12=highwaynet(input_tensor=L11,filters=None,kernel_size=3,
                     strides=1,padding="SAME",dilation_rate=1,
                     activation=None,dropout_rate=dropout_rate,scope_name="textencoder_highwaynet_Block"+str(10))
        

    L13=highwaynet(input_tensor=L12,filters=None,kernel_size=1,
                     strides=1,padding="SAME",dilation_rate=1,
                     activation=None,dropout_rate=dropout_rate,scope_name="textencoder_highwaynet_Block"+str(11))
    L14=highwaynet(input_tensor=L13,filters=None,kernel_size=1,
                     strides=1,padding="SAME",dilation_rate=1,
                     activation=None,dropout_rate=dropout_rate,scope_name="textencoder_highwaynet_Block"+str(12))
        
        
    K,V=tf.split(L14,2,axis=-1)
    return K,V
    

In [29]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

def audioencoder(input_tensor,dropout_rate,num_hidden_layers):
    L1=conv1d(input_tensor=input_tensor,filters=num_hidden_layers,kernel_size=1,strides=1,padding="CAUSAL",dilation_rate=1,
       activation=tf.nn.relu,dropout_rate=dropout_rate)
    
    L2=conv1d(input_tensor=L1,filters=None,kernel_size=1,strides=1,padding="CAUSAL",dilation_rate=1,
       activation=tf.nn.relu,dropout_rate=dropout_rate)
    
    L3=conv1d(input_tensor=L2,filters=None,kernel_size=1,strides=1,padding="CAUSAL",dilation_rate=1,
       activation=None,dropout_rate=dropout_rate)

    L4=highwaynet(input_tensor=L3,filters=None,kernel_size=3,
                         strides=1,padding="CAUSAL",dilation_rate=1,
                         activation=None,dropout_rate=dropout_rate,scope_name="audioencoder_highwaynet_Block"+str(1))
    L5=highwaynet(input_tensor=L4,filters=None,kernel_size=3,
                         strides=1,padding="CAUSAL",dilation_rate=3,
                         activation=None,dropout_rate=dropout_rate,scope_name="audioencoder_highwaynet_Block"+str(2))
    L6=highwaynet(input_tensor=L5,filters=None,kernel_size=3,
                         strides=1,padding="CAUSAL",dilation_rate=9,
                         activation=None,dropout_rate=dropout_rate,scope_name="audioencoder_highwaynet_Block"+str(3))
    L7=highwaynet(input_tensor=L6,filters=None,kernel_size=3,
                         strides=1,padding="CAUSAL",dilation_rate=27,
                         activation=None,dropout_rate=dropout_rate,scope_name="audioencoder_highwaynet_Block"+str(4))
    
    L8=highwaynet(input_tensor=L7,filters=None,kernel_size=3,
                         strides=1,padding="CAUSAL",dilation_rate=1,
                         activation=None,dropout_rate=dropout_rate,scope_name="audioencoder_highwaynet_Block"+str(5))
    L9=highwaynet(input_tensor=L8,filters=None,kernel_size=3,
                         strides=1,padding="CAUSAL",dilation_rate=3,
                         activation=None,dropout_rate=dropout_rate,scope_name="audioencoder_highwaynet_Block"+str(6))
    L10=highwaynet(input_tensor=L9,filters=None,kernel_size=3,
                         strides=1,padding="CAUSAL",dilation_rate=9,
                         activation=None,dropout_rate=dropout_rate,scope_name="audioencoder_highwaynet_Block"+str(7))
    L11=highwaynet(input_tensor=L10,filters=None,kernel_size=3,
                         strides=1,padding="CAUSAL",dilation_rate=27,
                         activation=None,dropout_rate=dropout_rate,scope_name="audioencoder_highwaynet_Block"+str(8))
    
    L12=highwaynet(input_tensor=L11,filters=None,kernel_size=3,
                     strides=1,padding="CAUSAL",dilation_rate=3,
                     activation=None,dropout_rate=dropout_rate,scope_name="audioencoder_highwaynet_Block"+str(9))
    L13=highwaynet(input_tensor=L12,filters=None,kernel_size=3,
                     strides=1,padding="CAUSAL",dilation_rate=3,
                     activation=None,dropout_rate=dropout_rate,scope_name="audioencoder_highwaynet_Block"+str(10))



    return L13
    

In [30]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

def audiodecoder(input_tensor,dropout_rate,num_hidden_layers,num_mels):
    L1=conv1d(input_tensor=input_tensor,filters=num_hidden_layers,kernel_size=1,strides=1,padding="CAUSAL",dilation_rate=1,
       activation=None,dropout_rate=dropout_rate)

    L2=highwaynet(input_tensor=L1,filters=None,kernel_size=3,
        strides=1,padding="CAUSAL",dilation_rate=1,
        activation=None,dropout_rate=dropout_rate,scope_name="audiodecoder_highwaynet_Block"+str(1))
    L3=highwaynet(input_tensor=L2,filters=None,kernel_size=3,
        strides=1,padding="CAUSAL",dilation_rate=3,
        activation=None,dropout_rate=dropout_rate,scope_name="audiodecoder_highwaynet_Block"+str(2))
    L4=highwaynet(input_tensor=L3,filters=None,kernel_size=3,
        strides=1,padding="CAUSAL",dilation_rate=9,
        activation=None,dropout_rate=dropout_rate,scope_name="audiodecoder_highwaynet_Block"+str(3))
    L5=highwaynet(input_tensor=L4,filters=None,kernel_size=3,
        strides=1,padding="CAUSAL",dilation_rate=27,
        activation=None,dropout_rate=dropout_rate,scope_name="audiodecoder_highwaynet_Block"+str(4))

    L6=highwaynet(input_tensor=L5,filters=None,kernel_size=3,
        strides=1,padding="CAUSAL",dilation_rate=1,
        activation=None,dropout_rate=dropout_rate,scope_name="audiodecoder_highwaynet_Block"+str(5))  
    L7=highwaynet(input_tensor=L6,filters=None,kernel_size=3,
        strides=1,padding="CAUSAL",dilation_rate=1,
        activation=None,dropout_rate=dropout_rate,scope_name="audiodecoder_highwaynet_Block"+str(6))
    
    L8=highwaynet(input_tensor=L7,filters=None,kernel_size=1,
                     strides=1,padding="CAUSAL",dilation_rate=1,
                     activation=tf.nn.relu,dropout_rate=dropout_rate,scope_name="audiodecoder_highwaynet_Block"+str(7))
    L9=highwaynet(input_tensor=L8,filters=None,kernel_size=1,
                     strides=1,padding="CAUSAL",dilation_rate=1,
                     activation=tf.nn.relu,dropout_rate=dropout_rate,scope_name="audiodecoder_highwaynet_Block"+str(8))
    L10=highwaynet(input_tensor=L9,filters=None,kernel_size=1,
                     strides=1,padding="CAUSAL",dilation_rate=1,
                     activation=tf.nn.relu,dropout_rate=dropout_rate,scope_name="audiodecoder_highwaynet_Block"+str(9))

        
    logits=conv1d(input_tensor=L10,filters=num_mels,kernel_size=1,strides=1,padding="CAUSAL",dilation_rate=1,
       activation=None,dropout_rate=dropout_rate)
    
    Y=tf.nn.sigmoid(logits)

    return logits,Y
    

In [31]:
import sys
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
sys.path.append('tools')
def attention(K,V,Q,num_hidden_units,mononotic_attention=False, prev_max_attentions=None):
    A=tf.matmul(Q,tf.transpose(K,[0,2,1]))/tf.sqrt(tf.to_float(num_hidden_units))
    A=tf.nn.softmax(A)
    R=tf.matmul(A,V)
    #they found it benefical for whatever reasons?
    R = tf.concat((R, Q), -1)
    #max_attentions = tf.argmax(A, -1)
    #alignments = tf.transpose(A, [0, 2, 1])
    return R,A
    

In [32]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

#gets mel as input (time/reduction factor) and spits out the upsampled meg 
def super_resolution(input_tensor,dropout_rate,num_hidden_layers,n_fft):
    L1=conv1d(input_tensor=input_tensor,filters=num_hidden_layers,kernel_size=1,strides=1,padding="SAME",dilation_rate=1,
       activation=None,dropout_rate=dropout_rate)

    L2=highwaynet(input_tensor=L1,filters=None,kernel_size=3,
                         strides=1,padding="SAME",dilation_rate=1,
                         activation=None,dropout_rate=dropout_rate,scope_name="SRNET_highwaynet_Block"+str(1))
    L3=highwaynet(input_tensor=L2,filters=None,kernel_size=3,
                         strides=1,padding="SAME",dilation_rate=3,
                         activation=None,dropout_rate=dropout_rate,scope_name="SRNET_highwaynet_Block"+str(2))
    
    L4=deconv(input_tensor=L3,filters=None,kernel_size=3,strides=2,
              padding='SAME',activation=None,dropout_rate=dropout_rate)    
    L5=highwaynet(input_tensor=L4,filters=None,kernel_size=3,
        strides=1,padding="SAME",dilation_rate=1,
        activation=None,dropout_rate=dropout_rate,scope_name="SRNET_highwaynet_Layer2"+str(3))
    L6=highwaynet(input_tensor=L5,filters=None,kernel_size=3,
        strides=1,padding="SAME",dilation_rate=3,
        activation=None,dropout_rate=dropout_rate,scope_name="SRNET_highwaynet_Layer2_1"+str(4))
        
    L7=deconv(input_tensor=L6,filters=None,kernel_size=3,strides=2,
              padding='SAME',activation=None,dropout_rate=dropout_rate)    
    L8=highwaynet(input_tensor=L7,filters=None,kernel_size=3,
        strides=1,padding="SAME",dilation_rate=1,
        activation=None,dropout_rate=dropout_rate,scope_name="SRNET_highwaynet_Layer2"+str(5))
    L9=highwaynet(input_tensor=L8,filters=None,kernel_size=3,
        strides=1,padding="SAME",dilation_rate=3,
        activation=None,dropout_rate=dropout_rate,scope_name="SRNET_highwaynet_Layer2_1"+str(6))
        
    L10=conv1d(input_tensor=L9,filters=2*num_hidden_layers,kernel_size=1,strides=1,padding="SAME",dilation_rate=1,
       activation=None,dropout_rate=dropout_rate)
        
    L11=highwaynet(input_tensor=L10,filters=None,kernel_size=3,
                         strides=1,padding="SAME",dilation_rate=1,
                         activation=None,dropout_rate=dropout_rate,scope_name="SRNET_highwaynet_Block"+str(7))
    
    L12=highwaynet(input_tensor=L11,filters=None,kernel_size=3,
                         strides=1,padding="SAME",dilation_rate=1,
                         activation=None,dropout_rate=dropout_rate,scope_name="SRNET_highwaynet_Block"+str(8))
        
    #????1+n_fft//2
    L13=conv1d(input_tensor=L12,filters=1+n_fft//2,kernel_size=1,strides=1,padding="SAME",dilation_rate=1,
       activation=None,dropout_rate=dropout_rate)
    
    L14=conv1d(input_tensor=L13,filters=None,kernel_size=1,strides=1,padding="SAME",dilation_rate=1,
             activation=tf.nn.relu,dropout_rate=dropout_rate)
    L15=conv1d(input_tensor=L14,filters=None,kernel_size=1,strides=1,padding="SAME",dilation_rate=1,
             activation=tf.nn.relu,dropout_rate=dropout_rate)
       

        
    logits=conv1d(input_tensor=L15,filters=None,kernel_size=1,strides=1,padding="SAME",dilation_rate=1,
       activation=None,dropout_rate=dropout_rate)
    
    Z=tf.nn.sigmoid(logits)

    return logits,Z
    

In [33]:
import sys
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
sys.path.append('tools')
sys.path.append('network')
from scipy.io.wavfile import write
import numpy as np
from tqdm import tqdm
import IPython


'''
if you're not sure how to structure your tensorFlow model especially when it comes to working with multiple graphs in TensorFlow I highly recommend reading the following blog posts,they help you to tease out the TensorFlow Graph Mess.(at least you wouldn't get lost when you look through your TensorBoard graph)

[1]https://danijar.com/structuring-your-tensorflow-models/
[2]https://gist.github.com/Breta01/cabbb5c7d9bbd3d9b4ec404828ac24bb

'''
class model(object):
    def __init__(self, data_path,mode):
        
        if mode=='training_superresolution':
            self.mels, self.mags, self.fnames,self.num_batch=get_batch_npz(data_path,
                                                                           'metadata.csv',HP.batch_size,HP.n_mels,HP.n_fft,mode=1)
            
            with tf.variable_scope('Super_Resolution_Network'):
                self.logits,self.Z=super_resolution(input_tensor=self.mels,dropout_rate=
                                                    HP.dropout_rate,num_hidden_layers=HP.c,n_fft=HP.n_fft)
                
        if mode=='training_text2sp':
            self.texts, self.mels, self.fnames,self.num_batch=get_batch_npz(data_path,
                                                                                  'metadata.csv',HP.batch_size,HP.n_mels,HP.n_fft,mode=2)
            self.L=embeding_layer(inputtextids=self.texts,emdeding_size=HP.embeding_num_units,
                             vocab_size=len(HP.persianvocab),scope_name="embeding")
            #one preview shifted target input
            self.S = tf.pad(self.mels[:,:-1,:],[[0,0],[1,0],[0,0]])

            with tf.variable_scope("Text_Encoder"):
                self.K,self.V = textencoder(embeding_tensor=self.L,dropout_rate=HP.dropout_rate,num_hidden_layers=HP.d)
            
            with tf.variable_scope("Audio_Encoder"):
                self.Q = audioencoder(input_tensor=self.S,dropout_rate=HP.dropout_rate,num_hidden_layers=HP.d)

            with tf.variable_scope("Attention"):
                 self.R,self.A = attention(self.K,self.V, self.Q,HP.d)

            with tf.variable_scope("Audio_Decoder"):
                self.logits,self.Y = audiodecoder(self.R,dropout_rate=HP.dropout_rate,num_hidden_layers=HP.d,num_mels=HP.n_mels) 
        if mode=='training_superresolution' or mode=='training_text2sp':    
            #In order to keep track of how far we are in the training, we use one of Tensorflow’s training utilities, the global_step.
            with tf.variable_scope("gs"):
                self.global_step = tf.Variable(0, name='global_step', trainable=False)
           
        if mode=='training_superresolution' or mode=='training_text2sp':    
            with tf.variable_scope('Loss_Operations'):
                if mode=='training_text2sp':
                    # mel L1 loss
                    self.l1_distance_loss= tf.reduce_mean(tf.abs(self.mels-self.Y ))
                    self.binary_divergence_loss= tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits
                                                                (logits=self.logits, labels=self.mels))
                
                    #attention loss(guided attention) Refer to page 3 on the paper
                    N, T = tf.cast(tf.shape(self.A)[1],tf.float32), tf.cast(tf.shape(self.A)[2],tf.float32)
                    W = tf.fill(tf.shape(self.A),0.0) 
                    W = W + tf.expand_dims(tf.range(N),1)/N - tf.expand_dims(tf.range(T),0)/T 
                    self.att_W = 1.0 - tf.exp(-tf.square(W)/(2*0.2)**2) 
                    self.loss_att = tf.reduce_mean(tf.multiply(self.A,self.att_W))

                    # total loss
                    self.total_loss = self.l1_distance_loss+self.binary_divergence_loss  + self.loss_att
                    tf.summary.scalar('l1_distance_loss', self.l1_distance_loss)
                    tf.summary.scalar('binary_divergence_loss', self.binary_divergence_loss)
                    tf.summary.scalar('loss_att', self.loss_att)
                    tf.summary.scalar('total_loss', self.total_loss)
                    tf.summary.image('mel', tf.expand_dims(tf.transpose(self.mels[:1], [0, 2, 1]), -1))
                    tf.summary.image('predicted_mel', tf.expand_dims(tf.transpose(self.Y[:1], [0, 2, 1]), -1))
                    tf.summary.image('plot_attention', tf.expand_dims(tf.transpose(self.A[:1], [0, 2, 1]), -1))
                
                if mode=='training_superresolution':
                    self.l1_distance_loss = tf.reduce_mean(tf.abs(self.mags-self.Z ))
                    self.binary_divergence_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits
                                                                 (logits=self.logits, labels=self.mags))
                    self.total_loss=self.l1_distance_loss+self.binary_divergence_loss
                    #tensorboard information
                    tf.summary.scalar('l1_distance_loss', self.l1_distance_loss)
                    tf.summary.scalar('binary_divergence_loss', self.binary_divergence_loss)
                    tf.summary.scalar('total_loss', self.total_loss)
                    tf.summary.image('mags', tf.expand_dims(tf.transpose(self.mags[:1], [0, 2, 1]), -1))
                    tf.summary.image('predicted_mag', tf.expand_dims(tf.transpose(self.Z[:1], [0, 2, 1]), -1))
        if mode=='training_superresolution' or mode=='training_text2sp':    
        
            # Define training step that minimizes the loss with the Adam optimizer
            with tf.variable_scope('optimizer_scope'):
                #for more information :
                #https://stackoverflow.com/questions/33919948/how-to-set-adaptive-learning-rate-for-gradientdescentoptimizer    
                step = tf.to_float(self.global_step + 1)
                lr=HP.init_learinig_rate * 4000.0**0.5 * tf.minimum(step * 4000.0**-1.5, step**-0.5)
                #optimizer = tf.train.RMSPropOptimizer(learning_rate=lr)
                #accordint to the ADAM paper Good default settings for the tested machine 
                #learning problems are α = 0.001,β1 = 0.9, β2 = 0.999             
                #and  = 10−8 but in my case it didn't work!
                optimizer = tf.train.AdamOptimizer(learning_rate=lr,beta1= 0.6,beta2=0.95)
                tf.summary.scalar("lr", lr)
                #do gradient cliping
                grts = optimizer.compute_gradients(self.total_loss)
                clipped = []
                for grad, var in grts:
                    grad = tf.clip_by_value(grad, -1., 1.)
                    clipped.append((grad, var))
                    
                self.train_operation = optimizer.apply_gradients(clipped, global_step=self.global_step)
        if mode=='demo':
            self.INP = tf.placeholder(tf.int32, shape=(None, None))
            self.mels = tf.placeholder(tf.float32, shape=(None, None, HP.n_mels))
            L=embeding_layer(inputtextids=self.INP,emdeding_size=HP.embeding_num_units,
                             vocab_size=len(HP.persianvocab),scope_name="embeding")
            with tf.variable_scope("Text_Encoder"):
                K,V = textencoder(embeding_tensor=L,dropout_rate=HP.dropout_rate,num_hidden_layers=HP.d)
                
            with tf.variable_scope("Audio_Encoder"):
                Q = audioencoder(input_tensor=self.mels,dropout_rate=HP.dropout_rate,num_hidden_layers=HP.d)

            with tf.variable_scope("Attention"):
                R,A = attention(K,V, Q,HP.d)
            with tf.variable_scope("Audio_Decoder"):
                Ylogit, self.Y = audiodecoder(R,dropout_rate=HP.dropout_rate ,num_hidden_layers=HP.d,num_mels=HP.n_mels) 
            with tf.variable_scope('Super_Resolution_Network'):
                logits,self.Z=super_resolution(input_tensor=self.Y ,dropout_rate=
                                                    HP.dropout_rate,num_hidden_layers=HP.c,n_fft=HP.n_fft)
            self.sess=tf.Session()
            self.sess.run(tf.global_variables_initializer())

            text2sp_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'Text_Encoder|Audio_Encoder|Audio_Decoder|embeding')

            superresolution_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'Super_Resolution_Network')

            sv1 = tf.train.Saver(var_list=text2sp_vars)
            sv1.restore(self.sess, tf.train.latest_checkpoint('logs/text-to-spec'))

            sv2 = tf.train.Saver(var_list=superresolution_vars)
            sv2.restore(self.sess, tf.train.latest_checkpoint('logs/super_resolution'))
            print('model loaded :)')
        tf.summary.merge_all()
    
    def predict(self,lines):
        lines = [item.replace("آ","آ").replace("أ","ا").replace("ئ","ی").replace("ؤ","و") for item in lines]
        Input_Text=load_data_synthesize(lines,HP.Max_Number_Of_Chars)
        predicted_mel = np.zeros((Input_Text.shape[0],HP.Max_Number_Of_MelFrames,HP.n_mels)) 
        predicted_mag = np.zeros((Input_Text.shape[0],HP.Max_Number_Of_MelFrames,HP.c+1))
        print('predicting ( . •́ _ʖ •̀ .) ...')
        #TODO:Implement mononotic attention
        for i in tqdm(range(1,HP.Max_Number_Of_MelFrames)):
            previous_slice = predicted_mel[:,:i,:]
            model_out = self.sess.run(self.Y,{
                          self.mels: previous_slice,self.INP: Input_Text})
            predicted_mel[:,i,:] = model_out[:,-1,:]
        _Z = self.sess.run(self.Z, {self.Y: predicted_mel})
        
        print('converting the generated spectrogram to audio ｖ(⌒ｏ⌒)ｖ♪ ...')
        for i, mag in enumerate(_Z):
            wav = spectrogram2wav(mag)
            write('generated_samples/' + "/{}.wav".format(i+1),HP.sr,wav.astype(np.float32))
        print('DONE!')

In [34]:
gr=model('null','demo')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: `tf.layers.conv1d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv1D` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/convolutional.py:294: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:95: UserWarning: `tf.layers.conv2d_transpose` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2DTranspose` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/convolutional.py:1736: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)


INFO:tensorflow:Restoring parameters from logs/text-to-spec/model_gs_140k
INFO:tensorflow:Restoring parameters from logs/super_resolution/model_gs_141k
model loaded :)


In [44]:
sentenses=[" سلام من امیرعلی گلی هستم "]

In [45]:
gr.predict(sentenses)

[' سلام من امیرعلی گلی هستم E']
predicting ( . •́ _ʖ •̀ .) ...


100%|██████████| 209/209 [00:39<00:00,  5.28it/s]


converting the generated spectrogram to audio ｖ(⌒ｏ⌒)ｖ♪ ...
DONE!


In [46]:
import IPython
IPython.display.Audio("generated_samples/1.wav")